# Detect the bottom dots on an image and use it to construct a spatial mapping

## Read, open and display the image

In [ ]:
# Load modules
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

# Read image
path = './../data/calibration/exampledots.tiff'

bitdepth = 16

# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)

# Show image
plt.figure()
plt.imshow(image*1,origin='lower',cmap='gray')
plt.show()

## Extract grid data from loaded image

In [ ]:
# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)

markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]

Nmarkers = len(np.unique(markers))
Ndots = int(13*7)
maximumSize = 50
minimumSize = 5

xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.show()

## Couple image to a spatial grid